Used to generate XGB predictions

In [18]:
from xgboost import XGBRegressor

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.model_selection import KFold
from sklearn.metrics import r2_score, mean_squared_error

import seaborn as sns
import shap
from xgb_params import xgb_params as params, num_top_features as top_n

In [19]:
params

{'n_estimators': 200,
 'device': 'cuda',
 'objective': 'reg:squarederror',
 'eval_metric': 'rmsle',
 'enable_categorical': True,
 'early_stopping_rounds': 20,
 'colsample_bytree': 0.7,
 'eta': 0.1,
 'gamma': 0,
 'max_depth': 6,
 'min_child_weight': 3.0,
 'reg_alpha': 93.0,
 'reg_lambda': 0.8685796539747039,
 'n_jobs': 4}

In [20]:
# File Paths
test_df_path = '../../Dataset/test.csv/test.csv'
process_train_path = './processed/processed_train_2.csv'
processed_test_path = './processed/processed_test_2.csv'
feats_path = './best_feats/price_index_feats.csv'
output_path = './output/xgb_pred.csv'


In [21]:
feats_df = pd.read_csv(feats_path)
feats = feats_df['col_name'].values.tolist()[:top_n]
processed_df = pd.read_csv(process_train_path)
processed_df = processed_df[feats+['price_doc']]
X = processed_df.drop(['price_doc'], axis=1)
y = processed_df['price_doc']
X = X[feats]

test_df = pd.read_csv(test_df_path)
processed_test_df = pd.read_csv(processed_test_path)
processed_test_df.head()

,id,full_sq,life_sq,floor,max_floor,build_year,num_room,kitch_sq,state,area_m,...,big_road1_1line_yes,railroad_1line_no,railroad_1line_yes,material_1.0,material_2.0,material_3.0,material_4.0,material_5.0,material_6.0,material_nan
0,30474,39.0,20.700000,2,9,1998.0,1,8.9,3.0,2.615514e+07,...,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
1,30475,79.2,34.404467,8,17,0.0,3,1.0,1.0,2.553630e+07,...,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
2,30476,40.5,25.100000,3,5,1960.0,2,4.8,2.0,9.946335e+06,...,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
3,30477,62.8,36.000000,17,17,2016.0,2,62.8,3.0,2.149409e+07,...,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
4,30478,40.0,40.000000,17,17,0.0,1,1.0,1.0,2.553630e+07,...,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0


In [22]:
def train(X, y, evalset=None):    
    model = XGBRegressor(
                    # verbosity=0,
                    n_estimators=params['n_estimators'],
                    device=params['device'],
                    objective=params['objective'],
                    eval_metric=params['eval_metric'],
                    enable_categorical=params['enable_categorical'],
                    early_stopping_rounds=params['early_stopping_rounds'] if evalset else None,

                    eta=params['eta'],
                    max_depth = params['max_depth'],
                    gamma = params['gamma'],
                    # reg_alpha = params['reg_alpha'],
                    min_child_weight=params['min_child_weight'],
                    colsample_bytree=params['colsample_bytree'],
                    n_jobs=params['n_jobs']
                )
    if not evalset:
        model.fit(X, y, verbose=True)
    else:
        model.fit(X, y, eval_set=evalset, verbose=True)

    return model

def find_best_model(processed_df):
    # Return best model from kfolds
    best_model = None
    min_loss = float('inf')
    cv = KFold(n_splits=10, shuffle=True, random_state=42)
    for fold, (train_idx, test_idx) in enumerate(cv.split(processed_df)):
        X_train = processed_df.iloc[train_idx]
        y_train = X_train["price_doc"]
        X_train.drop(["price_doc"], axis=1, inplace=True)

        X_val = processed_df.iloc[test_idx]
        y_val = X_val["price_doc"]
        X_val.drop(["price_doc"], axis=1, inplace=True)

        evalset = [(X_val, y_val)]
        model = train(X_train, y_train, evalset)

        pred = model.predict(X_val)
        loss = mean_squared_error(y_val, pred, squared=False)

        if loss < min_loss:
            min_loss = loss
            best_model = model
    
    return best_model, min_loss

In [23]:
model, loss = find_best_model(processed_df)

[0]	validation_0-rmsle:0.61009
[1]	validation_0-rmsle:0.59282



A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
is_categorical_dtype is deprecated and will be removed in a future version. Use isinstance(dtype, CategoricalDtype) instead
is_categorical_dtype is deprecated and will be removed in a future version. Use isinstance(dtype, CategoricalDtype) instead
is_categorical_dtype is deprecated and will be removed in a future version. Use isinstance(dtype, CategoricalDtype) instead
is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.Spars

[2]	validation_0-rmsle:0.57619
[3]	validation_0-rmsle:0.56244
[4]	validation_0-rmsle:0.55155
[5]	validation_0-rmsle:0.54386
[6]	validation_0-rmsle:0.53548
[7]	validation_0-rmsle:0.52847
[8]	validation_0-rmsle:0.52230
[9]	validation_0-rmsle:0.51799
[10]	validation_0-rmsle:0.51355
[11]	validation_0-rmsle:0.50959
[12]	validation_0-rmsle:0.50627
[13]	validation_0-rmsle:0.50337
[14]	validation_0-rmsle:0.50099
[15]	validation_0-rmsle:0.49919
[16]	validation_0-rmsle:0.49772
[17]	validation_0-rmsle:0.49577
[18]	validation_0-rmsle:0.49408
[19]	validation_0-rmsle:0.49260
[20]	validation_0-rmsle:0.49171
[21]	validation_0-rmsle:0.49036
[22]	validation_0-rmsle:0.48970
[23]	validation_0-rmsle:0.48852
[24]	validation_0-rmsle:0.48764
[25]	validation_0-rmsle:0.48673
[26]	validation_0-rmsle:0.48627
[27]	validation_0-rmsle:0.48581
[28]	validation_0-rmsle:0.48530
[29]	validation_0-rmsle:0.48480
[30]	validation_0-rmsle:0.48416
[31]	validation_0-rmsle:0.48375
[32]	validation_0-rmsle:0.48307
[33]	validation_

is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
is_categorical_dtype is deprecated and will be removed in a future version. Use isinstance(dtype, CategoricalDtype) instead
is_categorical_dtype is deprecated and will be removed in a future version. Use isinstance(dtype, CategoricalDtype) instead
is_categorical_dtype is deprecated and will be removed in a future version. Use isinstance(dtype, CategoricalDtype) instead

A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.Spars

[0]	validation_0-rmsle:0.59034
[1]	validation_0-rmsle:0.57371
[2]	validation_0-rmsle:0.55700
[3]	validation_0-rmsle:0.54342
[4]	validation_0-rmsle:0.53172
[5]	validation_0-rmsle:0.52348
[6]	validation_0-rmsle:0.51529
[7]	validation_0-rmsle:0.50805
[8]	validation_0-rmsle:0.50190
[9]	validation_0-rmsle:0.49729
[10]	validation_0-rmsle:0.49273
[11]	validation_0-rmsle:0.48875
[12]	validation_0-rmsle:0.48537
[13]	validation_0-rmsle:0.48243
[14]	validation_0-rmsle:0.48020
[15]	validation_0-rmsle:0.47806
[16]	validation_0-rmsle:0.47623
[17]	validation_0-rmsle:0.47418
[18]	validation_0-rmsle:0.47229
[19]	validation_0-rmsle:0.47091
[20]	validation_0-rmsle:0.46991
[21]	validation_0-rmsle:0.46873
[22]	validation_0-rmsle:0.46781
[23]	validation_0-rmsle:0.46687
[24]	validation_0-rmsle:0.46595
[25]	validation_0-rmsle:0.46517
[26]	validation_0-rmsle:0.46444
[27]	validation_0-rmsle:0.46383
[28]	validation_0-rmsle:0.46334
[29]	validation_0-rmsle:0.46299
[30]	validation_0-rmsle:0.46265
[31]	validation_0-

...

is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
is_categorical_dtype is deprecated and will be removed in a future version. Use isinstance(dtype, CategoricalDtype) instead
is_categorical_dtype is deprecated and will be removed in a future version. Use isinstance(dtype, CategoricalDtype) instead
is_categorical_dtype is deprecated and will be removed in a future version. Use isinstance(dtype, CategoricalDtype) instead

A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.Spars

[6]	validation_0-rmsle:0.55116
[7]	validation_0-rmsle:0.54428
[8]	validation_0-rmsle:0.53827
[9]	validation_0-rmsle:0.53352
[10]	validation_0-rmsle:0.52904
[11]	validation_0-rmsle:0.52532
[12]	validation_0-rmsle:0.52209
[13]	validation_0-rmsle:0.51902
[14]	validation_0-rmsle:0.51697
[15]	validation_0-rmsle:0.51511
[16]	validation_0-rmsle:0.51338
[17]	validation_0-rmsle:0.51163
[18]	validation_0-rmsle:0.51012
[19]	validation_0-rmsle:0.50901
[20]	validation_0-rmsle:0.50802
[21]	validation_0-rmsle:0.50696
[22]	validation_0-rmsle:0.50641
[23]	validation_0-rmsle:0.50567
[24]	validation_0-rmsle:0.50490
[25]	validation_0-rmsle:0.50429
[26]	validation_0-rmsle:0.50363
[27]	validation_0-rmsle:0.50336
[28]	validation_0-rmsle:0.50281
[29]	validation_0-rmsle:0.50242
[30]	validation_0-rmsle:0.50208
[31]	validation_0-rmsle:0.50154
[32]	validation_0-rmsle:0.50132
[33]	validation_0-rmsle:0.50098
[34]	validation_0-rmsle:0.50083
[35]	validation_0-rmsle:0.50057
[36]	validation_0-rmsle:0.50028
[37]	validat

is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
is_categorical_dtype is deprecated and will be removed in a future version. Use isinstance(dtype, CategoricalDtype) instead
is_categorical_dtype is deprecated and will be removed in a future version. Use isinstance(dtype, CategoricalDtype) instead
is_categorical_dtype is deprecated and will be removed in a future version. Use isinstance(dtype, CategoricalDtype) instead

A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.Spars

[4]	validation_0-rmsle:0.54208
[5]	validation_0-rmsle:0.53376
[6]	validation_0-rmsle:0.52543
[7]	validation_0-rmsle:0.51831
[8]	validation_0-rmsle:0.51208
[9]	validation_0-rmsle:0.50729
[10]	validation_0-rmsle:0.50278
[11]	validation_0-rmsle:0.49861
[12]	validation_0-rmsle:0.49510
[13]	validation_0-rmsle:0.49217
[14]	validation_0-rmsle:0.48975
[15]	validation_0-rmsle:0.48747
[16]	validation_0-rmsle:0.48575
[17]	validation_0-rmsle:0.48383
[18]	validation_0-rmsle:0.48199
[19]	validation_0-rmsle:0.48049
[20]	validation_0-rmsle:0.47936
[21]	validation_0-rmsle:0.47814
[22]	validation_0-rmsle:0.47740
[23]	validation_0-rmsle:0.47630
[24]	validation_0-rmsle:0.47509
[25]	validation_0-rmsle:0.47419
[26]	validation_0-rmsle:0.47336
[27]	validation_0-rmsle:0.47281
[28]	validation_0-rmsle:0.47200
[29]	validation_0-rmsle:0.47136
[30]	validation_0-rmsle:0.47067
[31]	validation_0-rmsle:0.47031
[32]	validation_0-rmsle:0.46989
[33]	validation_0-rmsle:0.46945
[34]	validation_0-rmsle:0.46914
[35]	validatio

is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
is_categorical_dtype is deprecated and will be removed in a future version. Use isinstance(dtype, CategoricalDtype) instead
is_categorical_dtype is deprecated and will be removed in a future version. Use isinstance(dtype, CategoricalDtype) instead
is_categorical_dtype is deprecated and will be removed in a future version. Use isinstance(dtype, CategoricalDtype) instead


In [24]:
print("Loss: ", loss)
pred = model.predict(processed_test_df[feats])

Loss:  2293061.899287826


is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
is_categorical_dtype is deprecated and will be removed in a future version. Use isinstance(dtype, CategoricalDtype) instead
is_categorical_dtype is deprecated and will be removed in a future version. Use isinstance(dtype, CategoricalDtype) instead
is_categorical_dtype is deprecated and will be removed in a future version. Use isinstance(dtype, CategoricalDtype) instead


In [25]:
# Save predictions
prediction_df = pd.DataFrame({
    'id': test_df['id'],
    'price_doc': pred
})

prediction_df.to_csv(output_path, index=False)